In [6]:
from datetime import datetime
import nasdaqdatalink
import quandl
import yfinance as yf
import pandas as pd
from fredapi import Fred
import time


In [2]:
import json


def GenerateHighchartVar(df, fieldX, fieldY):
    df.reset_index()
    init = '[' + '\n'
    s = ''
    
    for i in range(len(df)):
        s = s + '[' + str(df.loc[i, fieldX].timestamp() * 1000) + ',' + str(df.loc[i, fieldY]) + '],' + '\n'
    
    s = s[:-2]
    s = init + s + "]"
    
    return s

def generateJSONDataFile(fileName, content):
    with open("data/data_" + fileName+ ".json", "w") as outfile:
        outfile.write(content)
        
def generateMetadataFile(dict, fileName):
    with open('data/meta_' + fileName + '.js', 'w') as convert_file:
     convert_file.write('var meta = ')
     convert_file.write(json.dumps(dict))
     convert_file.write(';')



In [10]:
# Yahoo Data
# https://www.ssga.com/library-content/products/fund-docs/etfs/us/information-schedules/spdr-etf-listing.pdf

def getYahooData(tickerDict):
    asOfDateTime = datetime.now()
    asOfDateTimeStr = asOfDateTime.strftime("%d/%m/%Y %H:%M:%S")

    tickerList = list(tickerDict.keys())
    d = yf.download(tickerList)
    rawData = d['Adj Close'].reset_index()
    print('Successfully download Yahoo data')

    for t in tickerList:
        name = t.replace('^','').replace('=F','').replace('=X','').replace('DX-Y.NYB','DXY')
        #rawData = yf.download(t)
        #indexedData = rawData['Adj Close'].tail(950).reset_index()
        indexedData = rawData[['Date',t]]
        #indexedData = rawData['Adj Close'].reset_index()
        indexedData.columns = ['Date','Value']
        indexedData.dropna()
        highChartTS = GenerateHighchartVar(indexedData, 'Date','Value')
        generateJSONDataFile(name, highChartTS)
        
        meta = {'name': name,
                'displayName': tickerDict[t],
                'dataFrom': (indexedData.head(1)['Date'].item()).strftime('%m/%Y'),
                'dataTo': (indexedData.tail(1)['Date'].item()).strftime('%m/%Y'),
            'currentUpdate': (indexedData.tail(1)['Date'].item()).strftime('%d-%m-%Y'),
            'currentValue' : indexedData.tail(1)['Value'].item(),
            'minDate' : (indexedData.iloc[indexedData['Value'].idxmin(),:]['Date']).strftime('%d-%m-%Y'),
            'minValue' : indexedData.min()['Value'],
            'maxDate' : (indexedData.iloc[indexedData['Value'].idxmax(),:]['Date']).strftime('%d-%m-%Y'),
            'maxValue' : indexedData.max()['Value'],
            'lastUpdate' : asOfDateTimeStr,
            'source' : 'Internet',
            'dataFilename' : '/macroview/data/data_' + name + '.json'
            }
        
        generateMetadataFile(meta, name)
    print('Successfully generate Yahoo data')
    




tickers_WorldIndex = {'^GSPC' : 'SP500', 
                        '^DJI' : 'Dow Jones', 
                        '^IXIC' : 'Nasdaq', 
                        '^RUT' : 'Russell 2000', 
                        '^VIX' : 'VIX', 
                        '^FTSE' : 'FTSE 100',
                        '^STOXX' : 'STOXX 600', 
                        '^FCHI' : 'CAC 40', 
                        '^GDAXI' : 'DAX', 
                        '^IBEX' : 'IBEX 35', 
                        '^N225' : 'Nikkei 225', 
                        '000001.SS' : 'Shanghai Stock Exchange Composite Index',
                        '^KS11' : 'KOSPI Composite Index',
                        '^HSI' : 'Hang Seng Index'}

tickers_ccy = {'DX-Y.NYB' : 'USD', 'EURUSD=X' : 'EURUSD','JPY=X' : 'USDJPY','GBPUSD=X' : 'GBPUSD', 'AUDUSD=X' : 'AUDUSD', 'NZDUSD=X' : 'NZDUSD','CNY=X' : 'CNY','CAD=X' : 'USDCAD'}
tickers_commodities = {'GC=F' : 'Gold', 'SI=F' : 'Silver', 'CL=F' : 'Crude oil', 'ALI=F' : 'Aluminum', 'HG=F' : 'Copper', 'NG=F' : 'Natural Gas'}
tickers_treasury = {'ZT=F' : 'US 2-Year Note', 'ZN=F' : 'US 10-Year Note', 'ZB=F' : 'US Treasury'}
tickers_sector = {'XLC' : 'Communication Service (XLC)', 'XLP' : 'Consumer Staples (XLP)', 'XLY' : 'Consumer Discretionary (XLY)', 'XLE' : 'Energy (XLE)', 'XLF' : 'Financial (XLF)', 'XLV' : 'Health Care (XLV)', 'XLI' : 'Industrial (XLI)', 'XLB' : 'Materials (XLB)', 'XLRE' : 'Real Estate (XLRE)', 'XLK' : 'Technology (XLK)', 'XLU' : 'Utilities (XLU)'}
tickers_style = {'SPTM' : 'SP 1500','SPLG' : 'Large Cap','SPMD' : 'Mid Cap','SPSM'  : 'Small Cap','SPYG'  : 'Growth','SPYV' : 'Value','SPYD' : 'High Dividend Yield'}
tickers_arg = {'LE=F' : 'Live Cattle', 'KC=F' : 'Coffee', 'ZC=F' : 'Corn', 'CT=F' : 'Cotton', 'ZS=F': 'Soybean', 'SB=F' : 'Sugar', 'ZW=F' : 'Wheat'} 
#tickers_yahoo = {**tickers_WorldIndex, **tickers_ccy, **tickers_commodities, **tickers_treasury, **tickers_sector, **tickers_style} 

getYahooData(tickers_ccy)
getYahooData(tickers_commodities)
getYahooData(tickers_treasury)
getYahooData(tickers_sector)
getYahooData(tickers_style)
getYahooData(tickers_arg)




[*********************100%***********************]  49 of 49 completed
Successfully download Yahoo data
Successfully generate Yahoo data
[*********************100%***********************]  7 of 7 completed
Successfully download Yahoo data
Successfully generate Yahoo data


In [12]:
# Nasdaq Data
tickers_spRatio = {"MULTPL/SHILLER_PE_RATIO_MONTH" : 'Shiller PE Ratio',"MULTPL/SP500_DIV_YIELD_MONTH" : 'S&P500 Dividend Yield',"MULTPL/SP500_PE_RATIO_MONTH" : 'S&P 500 PE Ratio',"MULTPL/SP500_EARNINGS_YIELD_MONTH" : 'S&P 500 Earning Yield',"MULTPL/SP500_PBV_RATIO_QUARTER" : 'S&P 500 Price to Book Ratio',"MULTPL/SP500_PSR_QUARTER" : 'S&P 500 Price to Sales Ratio'}
tickers_worldInflationYoY = {"RATEINF/INFLATION_USA" : 'US Inflation',"RATEINF/INFLATION_GBR" : 'UK Inflation',"RATEINF/INFLATION_EUR" : 'Euro Area Inflation',"RATEINF/INFLATION_JPN" : 'Japan Inflation'}

tickers_allNasdaq = {**tickers_spRatio, **tickers_worldInflationYoY}

asOfDateTime = datetime.now()
asOfDateTimeStr = asOfDateTime.strftime("%d/%m/%Y %H:%M:%S")

for t in tickers_allNasdaq:
    rawData = quandl.get(t, authtoken="cza_RyNfSzs9o1Z2QBs4")
    #indexedData = rawData.tail(950).reset_index()
    indexedData = rawData.reset_index()
    highChartTS = GenerateHighchartVar(indexedData, 'Date','Value')
    generateJSONDataFile(t.replace('/','_'), highChartTS)
    
    meta = {'name': t.replace('/','_'),
            'displayName': tickers_allNasdaq[t],
            'dataFrom': (indexedData.head(1)['Date'].item()).strftime('%m/%Y'),
            'dataTo': (indexedData.tail(1)['Date'].item()).strftime('%m/%Y'),
        'currentUpdate': (indexedData.tail(1)['Date'].item()).strftime('%d-%m-%Y'),
        'currentValue' : indexedData.tail(1)['Value'].item(),
        'minDate' : (indexedData.iloc[indexedData['Value'].idxmin(),:]['Date']).strftime('%d-%m-%Y'),
        'minValue' : indexedData.min()['Value'],
        'maxDate' : (indexedData.iloc[indexedData['Value'].idxmax(),:]['Date']).strftime('%d-%m-%Y'),
        'maxValue' : indexedData.max()['Value'],
        'lastUpdate' : asOfDateTimeStr,
        'source' : 'Nasdaq',
        'dataFilename' : '/macroview/data/data_' + t.replace('/','_') + '.json'
        }
    
    generateMetadataFile(meta, t.replace('/','_'))


print('Successfully download Nasdaq data')



Successfully download Nasdaq data


In [14]:
# Fred Data

fred = Fred(api_key='d79cebb1e12819cd44ed96cc291f0f72')

def generateFredMeta(indexedData, name = '', displayName = ''):
    asOfDateTime = datetime.now()
    asOfDateTimeStr = asOfDateTime.strftime("%d/%m/%Y %H:%M:%S")
    meta = {'name': name,
            'displayName': displayName,
            'dataFrom': (indexedData.head(1)['Date'].item()).strftime('%m/%Y'),
            'dataTo': (indexedData.tail(1)['Date'].item()).strftime('%m/%Y'),
            'currentUpdate': (indexedData.tail(1)['Date'].item()).strftime('%d-%m-%Y'),
            'currentValue' : indexedData.tail(1)['Value'].item(),
            'minDate' : (indexedData.iloc[indexedData['Value'].idxmin(),:]['Date']).strftime('%d-%m-%Y'),
            'minValue' : indexedData.min()['Value'],
            'maxDate' : (indexedData.iloc[indexedData['Value'].idxmax(),:]['Date']).strftime('%d-%m-%Y'),
            'maxValue' : indexedData.max()['Value'],
            'lastUpdate' : asOfDateTimeStr,
            'source' : 'Fred',
            'dataFilename' : '/macroview/data/data_' + name + '.json'
            }
    return meta

In [15]:
coreCPI = fred.get_series('CPILFESL')
coreCPIYOY = coreCPI.pct_change(periods=12)*100
coreCPIYOY = coreCPIYOY.dropna()
coreCPIYOY_reset = coreCPIYOY.reset_index()
coreCPIYOY_reset.columns = ['Date','Value']
highChartTS = GenerateHighchartVar(coreCPIYOY_reset, 'Date','Value')
meta = generateFredMeta(coreCPIYOY_reset, 'coreCPIYOY', 'US Core CPI')
generateMetadataFile(meta, 'coreCPIYOY')
generateJSONDataFile('coreCPIYOY', highChartTS)

In [16]:
headlineCPI = fred.get_series('CPIAUCSL')
headlineCPIYOY = headlineCPI.pct_change(periods=12)*100
headlineCPIYOY = headlineCPIYOY.dropna()
headlineCPIYOY_reset = headlineCPIYOY.reset_index()
headlineCPIYOY_reset.columns = ['Date','Value']
highChartTS = GenerateHighchartVar(headlineCPIYOY_reset, 'Date','Value')
meta = generateFredMeta(headlineCPIYOY_reset, 'headlineCPIYOY', 'US Headline CPI')
generateMetadataFile(meta, 'headlineCPIYOY')
generateJSONDataFile('headlineCPIYOY', highChartTS)

In [17]:
#10y2y yield spread
series = 'T10Y2Y' 
df = fred.get_series(series)
df = df.dropna()
df_reset = df.reset_index()
df_reset.columns = ['Date','Value']
highChartTS = GenerateHighchartVar(df_reset, 'Date','Value')
meta = generateFredMeta(df_reset, series, 'US 10-2 Year Treasury Yield Spread')
generateMetadataFile(meta, series)
generateJSONDataFile(series, highChartTS)


In [18]:
#Real Yield
series = 'DFII10' 
df = fred.get_series(series)
df = df.dropna()
df_reset = df.reset_index()
df_reset.columns = ['Date','Value']
highChartTS = GenerateHighchartVar(df_reset, 'Date','Value')
meta = generateFredMeta(df_reset, series, 'US Real Yield')
generateMetadataFile(meta, series)
generateJSONDataFile(series, highChartTS)



In [19]:
#Nominal Yield
series = 'DGS10' 
df = fred.get_series(series)
df = df.dropna()
df_reset = df.reset_index()
df_reset = df_reset.dropna()
df_reset.columns = ['Date','Value']
highChartTS = GenerateHighchartVar(df_reset, 'Date','Value')
meta = generateFredMeta(df_reset, series, 'US Nominal Yield')
generateMetadataFile(meta, series)
generateJSONDataFile(series, highChartTS)


In [20]:
#Breakeven Rate
series = 'T10YIE' 
df = fred.get_series(series)
df = df.dropna()
df_reset = df.reset_index()
df_reset = df_reset.dropna()
df_reset.columns = ['Date','Value']
highChartTS = GenerateHighchartVar(df_reset, 'Date','Value')
meta = generateFredMeta(df_reset, series, 'US 10-Year Breakeven Inflation Rate')
generateMetadataFile(meta, series)
generateJSONDataFile(series, highChartTS)


In [1]:
series = 'A191RL1Q225SBEA' 
df = fred.get_series(series)
df = df.dropna()
df_reset = df.reset_index()
df_reset = df_reset.dropna()
df_reset.columns = ['Date','Value']
highChartTS = GenerateHighchartVar(df_reset, 'Date','Value')
meta = generateFredMeta(df_reset, series, 'US Real GDP Growth (Percent Change from Preceding Period, SAAR)')
generateMetadataFile(meta, series)
generateJSONDataFile(series, highChartTS)



NameError: name 'fred' is not defined

In [22]:
series = 'UNRATE' 
df = fred.get_series(series)
df = df.dropna()
df_reset = df.reset_index()
df_reset = df_reset.dropna()
df_reset.columns = ['Date','Value']
highChartTS = GenerateHighchartVar(df_reset, 'Date','Value')
meta = generateFredMeta(df_reset, series, 'US Unemployment Rate')
generateMetadataFile(meta, series)
generateJSONDataFile(series, highChartTS)


In [23]:
series = 'PAYEMS' 
df = fred.get_series(series)
df = df.dropna()
df_reset = df.reset_index()
df_reset = df_reset.dropna()
df_reset.columns = ['Date','Value']
highChartTS = GenerateHighchartVar(df_reset, 'Date','Value')
meta = generateFredMeta(df_reset, series, 'US Total Nonfarm')
generateMetadataFile(meta, series)
generateJSONDataFile(series, highChartTS)

print('Successfully download Fred data')

Successfully download Fred data


In [25]:
durableGoods = fred.get_series('UMDMNO')
durableGoodsYOY = durableGoods.pct_change(periods=12)*100
durableGoodsYOY = durableGoodsYOY.dropna()
durableGoodsYOY_reset = durableGoodsYOY.reset_index()
durableGoodsYOY_reset.columns = ['Date','Value']
highChartTS = GenerateHighchartVar(durableGoodsYOY_reset, 'Date','Value')
meta = generateFredMeta(durableGoodsYOY_reset, 'UMDMNO', 'US Durable Goods New Orders YoY')
generateMetadataFile(meta, 'durableGoodsYOY')
generateJSONDataFile('durableGoodsYOY', highChartTS)

## Testing

In [ ]:
# https://colab.research.google.com/drive/1S7bAG8HAJ6B-KkWlClmeOAc9V9HJOPZB?usp=sharing#scrollTo=ti8GH8YikINi

import yfinance as yf
import pandas as pd


In [ ]:
payload = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp500_table = payload[0]
sp500_table

In [ ]:
sp500_tickers = sp500_table.Symbol.str.upper().values
sp500_names = sp500_table.Security.values
sp500_sectors = sp500_table["GICS Sector"].values
sp500_sub_sectors = sp500_table["GICS Sub-Industry"].values

sp500_names_mapping = dict(zip(sp500_tickers, sp500_names))
sp500_sector_mapping = dict(zip(sp500_names, sp500_sectors))
sp500_sub_sector_mapping = dict(zip(sp500_names, sp500_sub_sectors))


In [ ]:
tickers = list(sp500_tickers)
prices = yf.download(tickers, start="2020-01-01", end="2021-12-31", interval='1d')
prices = prices["Adj Close"]
prices = prices.rename(columns=sp500_names_mapping)
prices

In [ ]:
prices.to_csv('sp500prices.csv')

In [ ]:
# https://www.ssga.com/library-content/products/fund-docs/etfs/us/information-schedules/spdr-etf-listing.pdf
tickers_WorldIndex = ['^GSPC','^DJI','^IXIC','^RUT','^VIX','^FTSE','^N225','^HSI']
tickers_ccy = ['EURUSD=X','JPY=X','GBPUSD=X', 'AUDUSD=X', 'NZDUSD=X','CNY=X','CAD=X']
tickers_commodities = ['GC=F','SI=F','CL=F']
tickers_treasury = ['ZT=F','ZN=F','ZB=F']

tickers_sector = ['XLC','XLP','XLY','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU']
tickers_style = ['SPTM','SPLG','SPMD','SPSM','SPYG','SPYV','SPYD']

prices = yf.download(tickers_WorldIndex+tickers_ccy+tickers_commodities+tickers_treasury+tickers_sector+tickers_style)
prices['Adj Close'].reset_index()


In [ ]:
prices['Adj Close'].to_csv('yahooData.csv')

In [ ]:
import investpy
data = investpy.economic_calendar(from_date='01/03/2022', to_date='01/09/2022')
data

In [ ]:
import dbnomics
worldGDP = dbnomics.fetch_series('WB', 'WDI', 'NY.GDP.MKTP.KD.ZG-1W')
worldGDP

In [ ]:
worldGDP[['period','value']]

In [ ]:
f = worldGDP[['period','value']]
dataFileName = 'worldGDP'
x = generateJSVar(f_reset, 'x1', 'period', isX=True)
y = generateJSVar(f_reset, 'data1','value', isX=False)
generateJSDataFile('data_'+ dataFileName,[x,y])

In [4]:
tsla = yf.Ticker("GC=F")
tsla.info

{'exchange': 'CMX',
 'shortName': 'Gold Jun 22',
 'exchangeTimezoneName': 'America/New_York',
 'exchangeTimezoneShortName': 'EDT',
 'isEsgPopulated': False,
 'gmtOffSetMilliseconds': '-14400000',
 'underlyingSymbol': 'GCM22.CMX',
 'quoteType': 'FUTURE',
 'symbol': 'GC=F',
 'underlyingExchangeSymbol': 'GCM22.CMX',
 'headSymbol': 'GC=F',
 'market': 'us24_market',
 'previousClose': 1937.8,
 'regularMarketOpen': 1934.3,
 'twoHundredDayAverage': 1822.8325,
 'trailingAnnualDividendYield': None,
 'payoutRatio': None,
 'volume24Hr': None,
 'regularMarketDayHigh': 1947.7,
 'navPrice': None,
 'averageDailyVolume10Day': 15159,
 'totalAssets': None,
 'regularMarketPreviousClose': 1937.8,
 'fiftyDayAverage': 1905.338,
 'trailingAnnualDividendRate': None,
 'open': 1934.3,
 'toCurrency': None,
 'averageVolume10days': 15159,
 'expireDate': 1656374400,
 'yield': None,
 'algorithm': None,
 'dividendRate': None,
 'exDividendDate': None,
 'beta': None,
 'circulatingSupply': None,
 'startDate': None,
 'reg

In [5]:
tsla.info['exchange']

'CMX'

In [6]:
etf = yf.Ticker("XLE")
etf.info

{'exchange': 'PCX',
 'shortName': 'SPDR Select Sector Fund - Energ',
 'longName': 'Energy Select Sector SPDR\xa0Fund',
 'exchangeTimezoneName': 'America/New_York',
 'exchangeTimezoneShortName': 'EDT',
 'isEsgPopulated': False,
 'gmtOffSetMilliseconds': '-14400000',
 'quoteType': 'ETF',
 'symbol': 'XLE',
 'messageBoardId': 'finmb_8795479',
 'market': 'us_market',
 'annualHoldingsTurnover': None,
 'enterpriseToRevenue': None,
 'beta3Year': 1.68,
 'profitMargins': None,
 'enterpriseToEbitda': None,
 '52WeekChange': None,
 'morningStarRiskRating': None,
 'forwardEps': None,
 'revenueQuarterlyGrowth': None,
 'fundInceptionDate': 913766400,
 'annualReportExpenseRatio': None,
 'totalAssets': 36983238656,
 'bookValue': None,
 'fundFamily': 'SPDR State Street Global Advisors',
 'lastFiscalYearEnd': None,
 'netIncomeToCommon': None,
 'trailingEps': None,
 'lastDividendValue': None,
 'SandP52WeekChange': None,
 'priceToBook': None,
 'nextFiscalYearEnd': None,
 'yield': 0.0392,
 'mostRecentQuarter

In [1]:
import pandas as pd
df = pd.read_csv(                    
	'https://www.econdb.com/api/series/RGDPUS/?format=csv',                    
	index_col='Date', parse_dates=['Date'])

In [2]:
df

,RGDPUS
Date,
1947-01-01,2034450
1947-04-01,2029024
1947-07-01,2024834
1947-10-01,2056508
1948-01-01,2087442
...,...
2020-10-01,18767778
2021-01-01,19055656
2021-04-01,19368310
